In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/churn_data.csv')

In [3]:
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [4]:
# clean data
# convert binaries
churn_dict = {False: 0, True: 1}
yes_no_dict = {'no': 0, 'yes': 1}
df['churn'].replace(churn_dict, inplace=True)
df['international plan'].replace(yes_no_dict, inplace=True)
df['voice mail plan'].replace(yes_no_dict, inplace=True)

In [5]:
df.pop('phone number')

0       382-4657
1       371-7191
2       358-1921
3       375-9999
4       330-6626
          ...   
3328    414-4276
3329    370-3271
3330    328-8230
3331    364-6381
3332    400-4344
Name: phone number, Length: 3333, dtype: object

In [6]:
df.churn.value_counts()

0    2850
1     483
Name: churn, dtype: int64

In [7]:
df.dtypes

state                      object
account length              int64
area code                   int64
international plan          int64
voice mail plan             int64
number vmail messages       int64
total day minutes         float64
total day calls             int64
total day charge          float64
total eve minutes         float64
total eve calls             int64
total eve charge          float64
total night minutes       float64
total night calls           int64
total night charge        float64
total intl minutes        float64
total intl calls            int64
total intl charge         float64
customer service calls      int64
churn                       int64
dtype: object

In [8]:
df.head()

,state,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [9]:
X = df.drop('churn', axis=1)
y = df.churn

In [10]:
df.pop('area code')

0       415
1       415
2       415
3       408
4       415
       ... 
3328    415
3329    415
3330    510
3331    510
3332    415
Name: area code, Length: 3333, dtype: int64

In [11]:
df.head()

,state,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [47]:
df.dtypes

state                      object
account length              int64
international plan          int64
voice mail plan             int64
number vmail messages       int64
total day minutes         float64
total day calls             int64
total day charge          float64
total eve minutes         float64
total eve calls             int64
total eve charge          float64
total night minutes       float64
total night calls           int64
total night charge        float64
total intl minutes        float64
total intl calls            int64
total intl charge         float64
customer service calls      int64
churn                       int64
dtype: object

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021, test_size=0.20)

In [13]:
from sklearn.preprocessing import OneHotEncoder

def ohe2021(X, column_list):  
    # One hot encode
    # get dataframe of categroical columns to oneHotEncode
    X_cat = X[column_list]
    X_cat.head()
    # get dataframe of numeric columns to oneHotEncode
    X_num = X.drop(column_list, axis=1)
    X_num.head()
    # Finally, One Hot Encode
    # from sklearn.preprocessing import OneHotEncoder
    ohe = OneHotEncoder(sparse=False, drop='first')
    X_cat_oh = pd.DataFrame(ohe.fit_transform(X_cat), index= X_cat.index, columns=ohe.get_feature_names(X_cat.columns))
    X_cat_oh.head()
    # sandwich OHE df with numerical dataframes
    X = X_num.join(X_cat_oh)
    return X
X_train = ohe2021(X_train, ['state'])

In [14]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=2021)
X_train_resamp, y_train_resamp = sm.fit_resample(X_train, y_train)

In [15]:
# # Scale * ONLY ON TRAINING DATA
from sklearn.preprocessing import StandardScaler
# scale data
def scale2021(X):
    ss = StandardScaler()
    X_scaled = pd.DataFrame(ss.fit_transform(X))
    X_scaled.index = X.index
    X_scaled.columns = X.columns
    return X_scaled
X_resamp_scaled = scale2021(X_train_resamp)
X_resamp_scaled.head()

,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,-1.287541,-0.665524,-0.323039,2.082042,0.918600,-0.731569,0.338874,-0.731237,2.091349,-0.850296,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
1,-0.654555,-0.496762,-0.323039,-0.480298,-0.534151,0.878502,-0.700258,0.878069,0.961682,-0.903917,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
2,0.743287,1.793591,-0.323039,-0.480298,-0.534151,-1.099726,-1.583519,-1.099876,1.297146,-0.045979,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,7.821370,-0.153545,-0.178571,-0.155816
3,2.615869,-0.496762,-0.323039,-0.480298,-0.534151,0.562705,0.494743,0.562368,1.871074,-0.743054,...,-0.128759,-0.137884,7.325934,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
4,-0.074319,-0.665524,-0.323039,2.082042,1.725684,0.150370,-0.648301,0.150416,1.165790,0.865579,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,7.328861


In [16]:
X_resamp_scaled.pop('area code')

0      -0.665524
1      -0.496762
2       1.793591
3      -0.496762
4      -0.665524
          ...   
4535    1.793591
4536   -0.496762
4537   -0.496762
4538   -0.496762
4539   -0.665524
Name: area code, Length: 4540, dtype: float64

In [17]:
X_resamp_scaled.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,-1.287541,-0.323039,2.082042,0.918600,-0.731569,0.338874,-0.731237,2.091349,-0.850296,2.091343,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
1,-0.654555,-0.323039,-0.480298,-0.534151,0.878502,-0.700258,0.878069,0.961682,-0.903917,0.962019,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
2,0.743287,-0.323039,-0.480298,-0.534151,-1.099726,-1.583519,-1.099876,1.297146,-0.045979,1.297250,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,7.821370,-0.153545,-0.178571,-0.155816
3,2.615869,-0.323039,-0.480298,-0.534151,0.562705,0.494743,0.562368,1.871074,-0.743054,1.870234,...,-0.128759,-0.137884,7.325934,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
4,-0.074319,-0.323039,2.082042,1.725684,0.150370,-0.648301,0.150416,1.165790,0.865579,1.166486,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,7.328861


In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.impute import MissingIndicator, SimpleImputer

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel

# plot_confusion_matrix is a handy visual tool, added in the latest version of scikit-learn
# if you are running an older version, comment out this line and just use confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [19]:
log = LogisticRegression()
log.fit(X_resamp_scaled, y_train_resamp)

LogisticRegression()

In [20]:
all_features_cross_val_score = cross_val_score(log, X_resamp_scaled, y_train_resamp, cv=3)
all_features_cross_val_score

array([0.72985469, 0.77197621, 0.76536682])

In [21]:
logreg_model = LogisticRegression(C=0.5, solver="liblinear")
logreg_model.fit(X_resamp_scaled, y_train_resamp)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_resamp_scaled, y_train_resamp, cv=3))

Old: [0.72985469 0.77197621 0.76536682]
New: [0.72919419 0.77263714 0.76404494]


In [22]:
logreg_model = LogisticRegression(class_weight='balanced', solver="liblinear", penalty='l1', C=0.5)
logreg_model.fit(X_resamp_scaled, y_train_resamp)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_resamp_scaled, y_train_resamp, cv=3))

Old: [0.72985469 0.77197621 0.76536682]
New: [0.7324967  0.76999339 0.76933245]


In [23]:
X_resamp_scaled.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,-1.287541,-0.323039,2.082042,0.918600,-0.731569,0.338874,-0.731237,2.091349,-0.850296,2.091343,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
1,-0.654555,-0.323039,-0.480298,-0.534151,0.878502,-0.700258,0.878069,0.961682,-0.903917,0.962019,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
2,0.743287,-0.323039,-0.480298,-0.534151,-1.099726,-1.583519,-1.099876,1.297146,-0.045979,1.297250,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,7.821370,-0.153545,-0.178571,-0.155816
3,2.615869,-0.323039,-0.480298,-0.534151,0.562705,0.494743,0.562368,1.871074,-0.743054,1.870234,...,-0.128759,-0.137884,7.325934,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,-0.155816
4,-0.074319,-0.323039,2.082042,1.725684,0.150370,-0.648301,0.150416,1.165790,0.865579,1.166486,...,-0.128759,-0.137884,-0.163381,-0.17728,-0.137183,-0.160521,-0.152343,-0.153545,-0.178571,7.328861


In [24]:
X_resamp_scaled.columns

Index(['account length', 'international plan', 'voice mail plan',
       'number vmail messages', 'total day minutes', 'total day calls',
       'total day charge', 'total eve minutes', 'total eve calls',
       'total eve charge', 'total night minutes', 'total night calls',
       'total night charge', 'total intl minutes', 'total intl calls',
       'total intl charge', 'customer service calls', 'state_AL', 'state_AR',
       'state_AZ', 'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE',
       'state_FL', 'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL',
       'state_IN', 'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD',
       'state_ME', 'state_MI', 'state_MN', 'state_MO', 'state_MS', 'state_MT',
       'state_NC', 'state_ND', 'state_NE', 'state_NH', 'state_NJ', 'state_NM',
       'state_NV', 'state_NY', 'state_OH', 'state_OK', 'state_OR', 'state_PA',
       'state_RI', 'state_SC', 'state_SD', 'state_TN', 'state_TX', 'state_UT',
       'state_VA', 'st

In [25]:
X_resamp_scaled_top5 = X_resamp_scaled[['voice mail plan', 'number vmail messages',
                                             'total day minutes', 'international plan',
                                             'customer service calls', 'total intl calls',
                                        'state_MS', 'state_MT', 'state_IL', 'state_VA']]

logreg_model = LogisticRegression(class_weight='balanced', solver="liblinear", penalty='l1', C=0.5)
logreg_model.fit(X_resamp_scaled_top5, y_train_resamp)

print("Old:", all_features_cross_val_score)
print("New:", cross_val_score(logreg_model, X_resamp_scaled_top5, y_train_resamp, cv=3))

Old: [0.72985469 0.77197621 0.76536682]
New: [0.72589168 0.75743556 0.76272307]


In [26]:
coef_dict = {}
for coef, feat in zip(logreg_model.coef_[0,:],X_resamp_scaled.columns):
    coef_dict[feat] = coef
    
df_cof = coef_dict

In [27]:
logreg_model.coef_

array([[-2.0693359 ,  1.5203763 ,  0.82919088,  0.40587381,  0.57583627,
        -0.35360712,  0.16899009,  0.16158181, -0.19320291, -0.16235827]])

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

sm = SMOTE()
ss = StandardScaler()
ohe = OneHotEncoder()
log = LogisticRegression()



In [ ]:
preprocessing = make_column_transformer((BinAge(), ['age']),
                                      (BinCapital(), ['capital_gain']),
                                      (BinCapital(), ['capital_loss']),
                                      (OneHotEncoder(),
                                       make_column_selector(dtype_include=object)),
                                      (StandardScaler(), ['hours_per_week']),
                                      remainder='drop')

In [70]:
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.model_selection import GridSearchCV

#pipe = some_pipeline(using sklearn, we need imblearn)

param_grid = {
    'randomforestclassifier__n_estimators': [100, 250, 500],
}

search = GridSearchCV(dt_pipeline, param_grid, n_jobs=-1)

preprocessing = make_column_transformer((OneHotEncoder(), make_column_selector(dtype_include=object)),
                                       (StandardScaler(), make_column_selector(dtype_include=np.number),
                                        SMOTE()))


#preprocessing = make_column_transformer((OneHotEncoder(), make_column_selector(dtype_include=object)),
                                                # StandardScaler(),
                                                # SMOTE(),
                                                # LogisticRegression())
preprocessing

ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734e2ac8>),
                                ('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734e2390>)])

In [61]:
preprocessing.fit_transform(X_train)

array([[-1.21592604, -0.68516656, -0.3280287 , ..., -0.15173593,
        -0.1858394 , -0.15302453],
       [-0.61363445, -0.51913513, -0.3280287 , ..., -0.15173593,
        -0.1858394 , -0.15302453],
       [ 0.71642615,  1.73414853, -0.3280287 , ..., -0.15173593,
        -0.1858394 , -0.15302453],
       ...,
       [-2.14445891, -0.68516656, -0.3280287 , ..., -0.15173593,
        -0.1858394 , -0.15302453],
       [-1.54216732, -0.68516656, -0.3280287 , ..., -0.15173593,
        -0.1858394 , -0.15302453],
       [-2.32012729, -0.51913513, -0.3280287 , ..., -0.15173593,
        -0.1858394 , -0.15302453]])

In [62]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

dt_pipeline = make_pipeline(preprocessing, DecisionTreeClassifier(random_state=2021))
rf_pipeline = make_pipeline(preprocessing, RandomForestClassifier(random_state=2021))

In [71]:
rf_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734ca6a0>),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734e2358>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=2021))])

In [75]:
param_grid = {
    'randomforestclassifier__n_estimators': [100, 250, 500],
}

search = GridSearchCV(rf_pipeline, param_grid, n_jobs=-1)

search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734ca6a0>),
                                                                        ('standardscaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734e2358>)])),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=2021))]),
             n_jobs=-1,
             param_grid={'randomforestclassifier__n_estimators': [100, 250,
    

In [77]:
search.__dict__

{'scoring': None, 'estimator': Pipeline(steps=[('columntransformer',
                  ColumnTransformer(transformers=[('onehotencoder',
                                                   OneHotEncoder(),
                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734ca6a0>),
                                                  ('standardscaler',
                                                   StandardScaler(),
                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734e2358>)])),
                 ('randomforestclassifier',
                  RandomForestClassifier(random_state=2021))]), 'n_jobs': -1, 'iid': 'deprecated', 'refit': True, 'cv': None, 'verbose': 0, 'pre_dispatch': '2*n_jobs', 'error_score': nan, 'return_train_score': False, 'param_grid': {'randomforestclassifier__n_estimators': [100,
   250,
   500]}, 'multimetric_': False, 'best_index_': 2,

In [79]:
search.best_params_

{'randomforestclassifier__n_estimators': 500}

In [80]:
search.best_score_

0.9332398760461244

In [81]:
search.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fee734ca630>),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fee72510c88>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=500, random_state=2021))])

In [67]:
cross_val_score(dt_pipeline, X_train, y_train, scoring='recall')

# New: [0.87318362 0.91275611 0.90680767]

array([0.65      , 0.6835443 , 0.6835443 , 0.74683544, 0.74683544])

In [68]:
cross_val_score(rf_pipeline, X_train, y_train, scoring='recall')

# New: [0.90356671 0.93588896 0.93258427]

array([0.475     , 0.58227848, 0.48101266, 0.59493671, 0.63291139])

In [39]:
X_train.head()

,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
561,53,408,0,1,18,146.8,107,24.96,310.0,84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1764,77,415,0,0,0,245.2,87,41.68,254.1,83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1733,130,510,0,0,0,124.3,70,21.13,270.7,99,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3168,201,415,0,0,0,225.9,110,38.40,299.1,86,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562,99,408,0,1,28,200.7,88,34.12,264.2,116,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [40]:
from sklearn.model_selection import train_test_split, cross_val_score

scores = cross_val_score(smoted_scaled_pipeline, X_train, y_train, cv=10)
scores

array([0.79400749, 0.73782772, 0.78651685, 0.73782772, 0.75280899,
       0.77902622, 0.77067669, 0.7593985 , 0.7518797 , 0.80827068])

In [32]:
smoted_scaled_pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('smote', SMOTE()),
                ('logisticregression', LogisticRegression())])

In [33]:
X_test_cat = X_test['state']


X_test_cat

1396    MD
286     MI
1277    FL
3001    IL
13      MT
        ..
2533    CO
2909    SC
3266    OK
1487    UT
3160    NH
Name: state, Length: 667, dtype: object

In [34]:
smoted_scaled_pipeline.score(X_test, y_test)

ValueError: could not convert string to float: 'MD'